In [1]:
import numpy as np
import pandas as pd 

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
from modules import one_hot ,wakachi, meishi,FastText_Vectrizer,rename_section,exam_preprosses

2022-11-25 11:58:26.881565: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-25 11:58:27.063685: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-25 11:58:27.063707: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-25 11:58:27.099026: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-25 11:58:28.019587: W tensorflow/stream_executor/platform/de

In [4]:
# データの修正

In [5]:
df = pd.read_csv('DATA20200401_20220331(修正後).csv', encoding='cp932', header=None)

In [6]:
# ラベルの間違い修正
df[3].unique()

array(['A1', 'A2', 'AB1', 'AB3', 'AB4', 'AB5', 'AB6', 'B1', 'B2', 'BL1',
       'C1', 'C2', 'C３', 'C3', 'C3.', 'DW', 'H1', 'H10', 'H12', 'H13',
       'H13 ', 'H14', 'H15', 'H16', 'H17', 'H18', 'H19', 'H2', 'H20',
       'H3', 'H4', 'H5', 'H6', 'H6\n', 'H7', 'H７', 'H7.', 'H8', 'H８',
       'H9', 'HS', 'K1', 'K2', 'L1', 'L2', 'L3', 'L4', 'L5', 'LI1',
       'LI1\n', 'LI2', 'LI3', 'LI3\n', 'LI4', 'N1', 'P1', 'PL1', 'R1',
       'R2', 'S1', 'S1\n', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S9', 'U1',
       'U2', 'U3', 'U4', 'U5', nan], dtype=object)

In [7]:
df[3]=df[3].replace(['C３', 'C3', 'C3.'], 'C3')
df[3]=df[3].replace(['H13', 'H13 '], 'H13')
df[3]=df[3].replace(['H6', 'H6\n'], 'H6')
df[3]=df[3].replace(['H7', 'H７', 'H7.'], 'H7')
df[3]=df[3].replace(['H8', 'H８'], 'H8')
df[3]=df[3].replace(['LI3\n'], 'LI3')
df[3]=df[3].replace(['S1', 'S1\n'], 'S1')
df[3]=df[3].replace(['LI1','LI1\n'], 'LI1')

In [8]:
df[df[3]=='H14']

,0,1,2,3,4,5,6,7,8
4063,11,女,耳鼻,H14,頭部(脳),嗅球プロトコル。先天性嗅覚障害です ３Tでお願いします。,くも膜のう胞,依頼通りに嗅球プロトコル,単純


In [9]:
df[2] = df[2].replace('H1', '脳外')

In [10]:
df[3] = df[3].replace('H14', 'N1')

In [11]:
del df[7]

In [12]:
df[df.isnull().any(axis=1)]

,0,1,2,3,4,5,6,8
6593,53,男,心外,H2,頭部(脳),NaN,脳梗塞疑い,単純
19336,14,男,小児,NaN,頚部,3月より右頚部リンパ節腫脹あり。増大傾向あり近医受診、近医で生検後、当院に5月22日受診。悪...,右頚部リンパ節腫瘍,単純＋造影Gd


In [13]:
df = df.dropna()

In [16]:
df.loc[df[2].str.contains('総診'), 2] = '救急'
df.loc[df[2].str.contains('ﾍﾟｲﾝ'), 2] = '麻酔'

In [17]:
df[3] = df[3].replace('S7', 'S6')
df[3] = df[3].replace('LI4', 'LI3')

In [18]:
df=df.rename(columns={0:'year',1:'sex',2:'section',3:'label',4:'position',5:'purpose',6:'diagnosis',8:'CE'})

In [19]:
df = one_hot(df, 'sex')

['男' '女']


In [20]:
CE_list = df['CE'].unique()
df['CE']=df['CE'].replace( list(filter(lambda x: '造影' in x,CE_list)), '造影')

In [21]:
df = one_hot(df, 'CE')

['単純' '造影']


In [22]:
position_list = df['position'].unique()

df['position']=df['position'].replace( list(filter(lambda x: '下肢血管' in x,position_list)), '下肢血管')
df['position']=df['position'].replace( list(filter(lambda x: '頚部' in x,position_list)), '頚部')
df['position']=df['position'].replace( list(filter(lambda x: '上肢' in x,position_list)), '上肢')
df['position']=df['position'].replace( list(filter(lambda x: 'MRdevice' in x,position_list)), 'MRdevice')
df['position']=df['position'].replace( list(filter(lambda x: '全脊椎' in x,position_list)), '全脊椎')
df['position']=df['position'].replace( list(filter(lambda x: '腹部血管' in x,position_list)), '腹部血管')
df['position']=df['position'].replace( list(filter(lambda x: '下垂体' in x,position_list)), '下垂体')
df['position']=df['position'].replace( list(filter(lambda x: '乳房' in x,position_list)), '乳房')
df['position']=df['position'].replace( list(filter(lambda x: '乳房' in x,position_list)), '乳房')



df['position']=df['position'].replace( ['縦隔','胸部(郭)'], '胸郭')
df['position']=df['position'].replace( ['尿路系MR[VCG(単純)]', '泌尿器系下腹部'], '泌尿器系下腹部')
df['position']=df['position'].replace( ['下腹部委任', '下腹部(一般)'], '下腹部')

In [23]:
#条件にマッチしたIndexを取得
drop_index = df.index[(df['position'] == 'MRdevice') | 
                      (df['position'] == 'WB MRI') | 
                      (df['label'] == 'DW') 
                     ]
#drop_index + df.index[df['position'] == 'WB MRI']

print(drop_index)
#条件にマッチしたIndexを削除
df = df.drop(drop_index)

Int64Index([ 1371,  1372,  2589,  2887,  2888,  2953,  2954,  2955,  2956,
             2957,  4713,  4714,  4715,  4716,  4717,  4718,  4719,  4720,
             4721,  4722,  4723,  4724,  4725,  4726,  8156, 10046, 10047,
            10048, 10049, 10050, 10051, 10052, 10053, 10054, 10055, 10056,
            10057, 10058, 10059, 10060, 11735, 12277, 12531, 12776, 12777,
            13803, 13804, 13805, 13806, 17354, 17492, 17493, 17494, 17495,
            17496, 17497, 18396],
           dtype='int64')


In [24]:
df['position'].value_counts()

頭部(脳)        4675
頭部(委任)       3608
胆道系(MRCP)    1577
婦人科下腹部       1171
腰椎(髄)         980
上腹部(肝胆膵他）     753
下垂体           556
頚椎(髄)         529
膝関節           454
MRA脳          394
全脊椎           392
骨盤部           360
前立腺           358
肩関節           353
頚部            289
大腿部           266
乳房            241
内耳(後頭蓋窩)      202
上下顎部          186
胸椎(髄)         166
眼窩            123
股関節           111
胸郭            109
副鼻腔           106
泌尿器系下腹部       102
腎臓            101
上腹部委任          98
膀胱             90
下腹部            83
手及び手指          69
上腕部            67
足及び足指          65
胸部委任           56
心臓             54
下腿部            52
手関節            44
顎関節            43
足関節            40
脊椎(委任)         39
胎児撮影(AFG)      35
前腕部            28
肩軟部            28
膝軟部            27
腸管(経口法)        25
腎動脈（単純)        25
肘関節            24
副腎             20
下肢血管           20
下肢(委任)         20
手首軟部           14
上肢             13
肘軟部            11
腹部血管           10
足首軟部            9
直腸              7
Name: posi

In [25]:
df['label'].value_counts()

H2     3021
H7     1650
LI2    1633
H6     1102
S4      898
AB4     783
H5      767
B1      737
H1      663
H8      589
LI3     526
N1      495
S1      476
P1      405
L2      403
AB3     375
S5      369
H4      344
U2      333
H10     324
H19     272
B2      263
C3      243
H9      222
LI1     205
H3      196
S6      168
R1      139
K1      137
S3      137
H13     122
C1      110
BL1     108
H18     101
AB1      89
L3       81
AB5      74
L1       69
H15      65
R2       63
H12      60
C2       54
H17      49
S2       44
PL1      41
K2       39
L4       36
U3       35
HS       30
L5       26
AB6      25
A2       20
H20      19
U5       10
S9       10
U4        9
H16       6
U1        5
A1        3
Name: label, dtype: int64

In [26]:
df['label'] = df['label'].replace({'H14':'N1'})

In [27]:
df['label'].unique()

array(['A1', 'A2', 'AB1', 'AB3', 'AB4', 'AB5', 'AB6', 'B1', 'B2', 'BL1',
       'C1', 'C2', 'C3', 'H1', 'H10', 'H12', 'H13', 'N1', 'H15', 'H16',
       'H17', 'H18', 'H19', 'H2', 'H20', 'H3', 'H4', 'H5', 'H6', 'H7',
       'H8', 'H9', 'HS', 'K1', 'K2', 'L1', 'L2', 'L3', 'L4', 'L5', 'LI1',
       'LI2', 'LI3', 'P1', 'PL1', 'R1', 'R2', 'S1', 'S2', 'S3', 'S4',
       'S5', 'S6', 'S9', 'U1', 'U2', 'U3', 'U4', 'U5'], dtype=object)

In [28]:
df['section'].value_counts()

脳外    4524
整形    2558
脳内    2301
消内    1954
母女    1210
小児     730
呼内     663
泌尿     662
消外     576
耳鼻     569
呼外     545
放科     493
皮膚     385
乳一     330
精神     279
救急     234
循内     214
口外     213
眼科     185
血内     176
内泌     153
心外     103
腎内      75
麻酔      69
形外      48
腫内      29
Name: section, dtype: int64

In [29]:
df.isnull().all(axis=0)

year         False
section      False
label        False
position     False
purpose      False
diagnosis    False
男            False
女            False
単純           False
造影           False
dtype: bool

In [30]:
df.to_csv("CorrectedData.csv",index=False)

In [6]:
df = pd.read_csv("CorrectedData.csv")

In [8]:
len(df)

19278